In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten
using Random, Distributions
Random.seed!(5)

┌ Info: Precompiling JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1186
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/ec2-user/.julia/compiled/v1.1/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1184


MersenneTwister(UInt32[0x00000005], Random.DSFMT.DSFMT_state(Int32[-85655800, 1072952617, -522203688, 1073721902, 1588056292, 1072823262, 1930843968, 1073164255, -719121749, 1073022455  …  647343590, 1073165527, 1405924594, 1072769931, 1969171087, 42945320, -226019479, -1469955508, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000000

In [2]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")
include("decoder_utils.jl")

collectygold (generic function with 1 method)

In [3]:
d = JLD.load("english_chmodel3.jld2")
d["new_d2"]["unkchar"] = Char(0x11)
d["new_d2"]["sowchar"] = Char(0x12)
d["new_d2"]["eowchar"] = Char(0x13)

'\x13': ASCII/Unicode U+0013 (category Cc: Other, control)

In [4]:
v = create_vocab(d["new_d2"])
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d["new_d2"])

9-element Array{KnetArray{Float32,2},1}:
 KFloat32[0.593419 0.190279 … 0.11003 -0.000173569; -0.236287 0.455614 … 0.138825 -0.0971059; … ; -0.341316 0.0835031 … -0.0285478 -0.0718531; -0.205958 -0.168918 … -0.13226 -0.0319474]
 KFloat32[0.0708071 -0.456333 … 0.557484 0.094816; -1.61625 -1.37924 … 0.342474 0.548784; … ; 1.07664 -0.430524 … -0.360356 0.668587; 0.324491 -0.175173 … -0.790751 1.16417]            
 KFloat32[1.2185; 1.52402; … ; -1.1745; -1.53404]                                                                                                                                        
 KFloat32[1.47578 0.055072 … -0.2823 -1.58338; -1.95253 -0.402249 … -0.221281 -0.667834; … ; 1.28302 -0.0459206 … -2.12551 -0.95058; 0.924036 0.269601 … 0.515548 7.38111]               
 KFloat32[-0.916012; -2.25621; … ; -0.166855; 0.169325]                                                                                                                                  
 KFloat32[1.70814 0.787587 … 

In [5]:
s = corpus[1]

Sanal parçacıklar sa bunların hiçbirini yapamazlar . 

In [6]:
ev = extend_vocab!(v, corpus)
fs = createcolvecs(corpus,ev)

FeatureSource([-0.542853 -0.992666 … 1.6983 -2.17757; 0.584793 1.14789 … 0.609551 -2.53964; … ; -0.825722 -2.10416 … 1.05649 -0.0918268; -2.90343 -0.18003 … -0.493267 1.06719], [1.46431 -0.123083 … 0.537261 0.982212; 0.260871 0.318797 … 0.798075 0.462872; … ; 1.29726 0.789473 … -0.589126 0.00480883; -1.61916 1.94446 … -1.57499 -1.55228], [0.819346 0.699932 … -0.915596 -1.791; 0.504342 -2.51908 … -0.0451559 0.201541; … ; 0.493422 0.875163 … -0.962536 1.63154; 0.395833 0.817469 … -1.82224 0.441632])

In [7]:
fillvecs!(wmodel, corpus, v, fs)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{KnetArray{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:63
└ @ Main /home/ec2-user/GBDP/encoder_utils.jl:63


In [8]:
fillcavec!(corpus)


In [9]:
function non_lin(w,x)
   return transpose(w[1:length(w)-1])*x .+ w[length(w)] 
end

non_lin (generic function with 1 method)

In [17]:
bitches = minibatch(corpus, 10)

369-element Array{Any,1}:
 Any[.. , .. , ... , . , . , ... , Kanamıyordu ... , Gülümsemeler . , Niye ? , Tiyatrocular . ]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
# Going into autograd

In [44]:
function mynll(y,a::AbstractArray{<:Integer}; dims=1, average=true)
    indices = Knet.findindices(y,a,dims=dims)
    for i in 1:length(a)
       if (a[i]==0)
           deleteat!(indices, i) 
            break
        end
    end
    lp = logp(y,dims=dims)[indices]
    average ? -mean(lp) : -sum(lp)
end

mynll (generic function with 1 method)

In [45]:
# ******
struct Linear; w; b; end
(m::Linear)(x) = m.w * x .+ m.b
(m::Linear)(x, y) = mynll(reshape(m(x), (length(y), length(y))), y)
#(m::Linear)(data::Data) = mean(m(x,y) for (x,y) in data)
Linear(i::Int,o::Int,scale=0.01) = Linear(Param(KnetArray(map(Float32,scale * randn(o,i)))), Param(KnetArray(map(Float32,zeros(o)))))

Linear

In [48]:
# ***
model = Linear(length(corpus[1].cavec[1]) * 2, 1)

Linear(P(KnetArray{Float32,2}(1,1900)), P(KnetArray{Float32,1}(1)))

In [54]:
function sgdupdate!(func, args; lr=0.0005)
    fval = @diff func(args...)
    for param in Knet.params(fval)
        ∇param = grad(fval, param)
        param .-= lr * ∇param
    end
    return value(fval)
end

sgdupdate! (generic function with 1 method)

In [59]:
epoch = 20

20

In [60]:
for e in 1:epoch
   for s in corpus #b
        if length(s)>2
            println(s.word)
            rm = createrelmatrix(s)
            println(size(rm))
            loss = sgdupdate!(model, (rm, s.head))
            println(loss) 
        end
    end 
    #sleep(10)
end

["Sanal", "parçacıklar", "sa", "bunların", "hiçbirini", "yapamazlar", "."]
(1900, 49)
1.1891226
["Ona", "her", "şeyimi", "verdim", "."]
(1900, 25)
0.8219378
["Karşısında", ",", "pantolonu", "dizlerine", "dek", "ıslak", ",", "önlük", "torbası", "ham", "eriklerle", "dolu", "İbrahim", "dikiliyordu", "."]
(1900, 225)
1.9271133
["Nereden", "biliyorsunuz", "."]
(1900, 9)
0.12352967
["Aradığımı", "Buldum", "Sandım", "."]
(1900, 16)
1.5837054
["Kahveleri", "bende", "içelim", "."]
(1900, 16)
0.4768926
["Ne", "diyeceğimi", "bilemedim", "."]
(1900, 16)
0.8394658
["Süpürgen", "nerede", "."]
(1900, 9)
0.61267745
["Bir", "taksi", "bulduk", "."]
(1900, 16)
0.68196476
["Mebrure", "birden", "heyecanlanmıştı", "."]
(1900, 16)
0.45225534
["\"", "Burada", "ki", "üst", "düzey", "görüşmelerimizde", "turizm", "için", "ellerinden", "geleni", "yapacaklarını", "söylediler", "."]
(1900, 169)
2.234564
["Gel", "bak", ",", "sana", "evi", "göstereyim", "."]
(1900, 49)
1.4048713
["Kendileriyle", "görüştüğümden", "edi

3.0027876
["Başını", "kaşıyor", ",", "karar", "sız", "."]
(1900, 36)
1.1307678
["Vallahi", "çok", "şirin", "."]
(1900, 16)
0.43537268
["Paşa", ",", "dalkavuk", "arıyor", "."]
(1900, 25)
0.37818623
["Bu", "beni", "çok", "üzdü", "."]
(1900, 25)
0.6551422
["Taşhan'ın", "kapısının", "önünde", "bir", "an", "bir", "kararsızlık", "geçirdi", "."]
(1900, 81)
1.5973762
["Türkiye'ye", "Çin'den", "elli", "bin", "turist", "geleceğini", "öngörüyorum", "."]
(1900, 64)
1.6144236
["Geceyi", "böylece", "geçirebilecektik", "."]
(1900, 16)
0.38614845
["Gülmeyin", ",", "zor", "durum", "..."]
(1900, 25)
0.61521626
["Dirisi", "kanlı", "."]
(1900, 9)
0.22423816
["Kendine", "sakladığı", "bir", "gizemi", "var", "dır", "."]
(1900, 49)
1.4616283
["Yüksek", "teknolojinin", "yetersiz", "kaldığı", "bu", "havanın", ",", "bu", "iklimin", "içinde", "yetişen", "canlılar", ",", "birbirlerine", "ilaç", ",", "barınak", ",", "korunak", "oluyordu", "."]
(1900, 441)
2.5949948
["Hızlı", "adımlarla", "yeniden", "Taşhan'a", "dön

(1900, 900)
2.7542262
["Görüldüğü", "gibi", ",", "dışarıdan", "bakıldığında", "doktor", "baba", ",", "sadık", "eş", "ve", "üniversite", "bitirmiş", ",", "iş", "hayatına", "atıl", "mış", "delikanlı", ";", "toplumun", "aile", "imajına", "ve", "değerlerine", "ne", "de", "uygun", "bir", "aile", "."]
(1900, 961)
3.0474803
["Ancak", "burada", "üretilen", "elektron", "-", "pozitron", "çifti", "hiç", "yoktan", "yaratılmamış", ",", "gama", "fotonundan", "üretilmiştir", "."]
(1900, 225)
2.4425921
["Oh", ",", "iyi", "ki", "aradın", "Maryline", "."]
(1900, 49)
1.5486583
["Tahvilin", "ikinci", "kuponu", "ve", "bu", "kupona", "ilişkin", "yapılacak", "olan", "referans", "bono", "ihalesi", "ise", "doksan", "gün", "vade", "li", "olacak", "."]
(1900, 361)
2.5087764
["Hem", "emin", "im", "ki", ",", "benim", "de", "benden", "daha", "büyük", "bir", "kardeşim", "olsaydı", ",", "o", "da", "aynılarını", "yapardı", "."]
(1900, 361)
2.2167485
["Her", "gün", "tekrarlanan", "bu", "rutine", "belli", "ki", "onlar",

(1900, 441)
2.03142
["Hiç", "anlamasalar", "da", ",", "hiç", "bilmeseler", "de", ",", "hiç", "ilgilenmeseler", "de", ",", "neden", ",", "kadınlar", "bütün", "erkeklerin", "iş", "hayatlarını", "pürdikkat", "dinlerler", "."]
(1900, 484)
2.1168585
["Bir", "cevap", "geliyor", ":", "Gururlu", "ve", "onur", "lu", "."]
(1900, 81)
1.5003966
["Ama", "Farsçasının", "da", "kökeni", "Grekçe", "."]
(1900, 36)
1.4950695
["Aldırma", "sen", "bana", "."]
(1900, 16)
0.73251224
["Kim", "o", "?"]
(1900, 9)
0.22312927
["Tibet'in", "tek", "hayranı", "ben", "değilim", "."]
(1900, 36)
1.9014336
["Yine", "de", "iyi", "çocuklar", "mış", ",", "seni", "eve", "kadar", "getirmiş", "."]
(1900, 121)
1.7428089
["Koyu", "badem", "göz", "lü", ",", "koyu", "gür", "sakallı", "olur", "Tatarlar", "."]
(1900, 121)
1.4351591
["Onun", "yerine", "okula", ",", "'", "Adını", "mezar", "taşına", "değil", ",", "okula", "yazdır", "'", "pankartı", "asıldı", "."]
(1900, 256)
2.0614576
["Bir", "kutudan", "eski", "bir", "müzik", "seti", 

(1900, 289)
2.8604846
["'", "Gerisi", "de", "gelecek", "'", "."]
(1900, 36)
1.1529301
["Yanımda", "ki", "koltuğa", "yerleştirdi", "ince", "bedenini", "."]
(1900, 49)
1.4723371
["Bu", "konuda", "Cumhurbaşkanı", "Ahmet", "Necdet", "Sezer'in", "de", "\"", "Türban", "tartışması", "bitmiştir", "\"", "diyerek", "son", "noktayı", "koyduğu", "belirtildi", "."]
(1900, 324)
2.4204597
["Kafası", "karışık", "tı", "besbelli", "."]
(1900, 25)
3.108353
["Ama", ",", "dedi", "umutsuzlukla", "kadın", ",", "O", "beni", "düşünmediği", "sürece", "hiçbir", "şey", "yapamam", "ki", "!"]
(1900, 225)
2.34744
["Almanlar", "için", "."]
(1900, 9)
0.16726875
["Buna", "göre", "bakanlık", "soruşturmayı", "yenileyebilirdi", "."]
(1900, 36)
0.8748668
["Çayırlığa", "çıktılar", "."]
(1900, 9)
0.32791537
["Birçok", "durumda", "onların", "satın", "almayacakları", "birşeye", "bakmalarını", "sağlamak", "çok", "zor", "dur", "."]
(1900, 144)
2.170023
["Tek", "başımıza", "iktidara", "doğru", "yürüyoruz", "\"", "dedi", "."]
(190

...

["Bu", "dönemde", ",", "teori", ",", "bilimin", "çeşitli", "alanlarında", ",", "yeni", "kavramsal", "yapılanmalarla", "bilimi", "yeni", "düzlemlere", "taşımış", ";", "geçmişin", "kavramsal", "yapılanmalarında", "köklü", "bir", "kopuşla", "bilimin", "önünde", "yeni", "ufuklar", "açmıştır", "."]
(1900, 841)
2.5830216
["Fotokopi", "çekme", "yasağı", "kondu", "."]
(1900, 25)
1.0679657
["Kerem'in", "sözünü", "ettiği", "fotoğraf", "olmalıydı", "."]
(1900, 36)
1.3594823
["Gözüm", "hep", "onda", "dır", "."]
(1900, 25)
0.55288625
["Yeni", "bir", "sosyolojik", "tabana", "hitap", "eden", "bir", "parti", "yiz", "."]
(1900, 100)
1.770713
["Dedemin", "anlattıkları", "bana", "tarih", "bilgisi", "gibi", "geliyor", "."]
(1900, 64)
1.146284
["Dün", "gece", "diskoda", "unutmuşsun", "."]
(1900, 25)
1.1060811
["O", "zaman", ",", "klasik", "şube", "vardı", "."]
(1900, 49)
1.3991641
["Neniz", "var", "."]
(1900, 9)
0.3840121
["Olayın", "ilk", "günü", "düzenlenmiş", "bir", "rapor", "var", "örneğin", "."]
(1900

["Can", "sıkıcı", "bir", "iş", "."]
(1900, 25)
1.125535
["Sanırım", "gezmediği", "ülke", ",", "danışmanı", "olmadığı", "kimse", ",", "bilmediği", "ilke", "kalmadı", "."]
(1900, 144)
2.1603043
["Ama", "o", "kadar", "..."]
(1900, 16)
2.2244635
["Elini", "sol", "cebine", "sokup", "bir", "fotoğraf", "çıkarttı", ",", "uzun", "uzun", "baktı", "."]
(1900, 144)
2.1315181
["Türkiye", ",", "geçmişle", "ve", "geçmişin", "korkularıyla", "bağlarını", "kopardı", "gidiyor", "."]
(1900, 100)
2.1422586
["Adamın", "yanında", "ki", "masaya", "oturdum", "."]
(1900, 36)
1.3005458
["Kavurup", "leblebi", "yaparlar", "."]
(1900, 16)
0.4131629
["Bu", "incelemenin", "neticesinde", "vardıkları", "sonuç", ",", "uçak", "buzlanmadan", "pervanelerde", "ki", "ve", "motorda", "ki", "buzlanmadan", "dolayı", "düşmüş", "olabilir", "."]
(1900, 324)
2.496669
["Kızını", "çok", "beğendik", "."]
(1900, 16)
0.1666019
["Dalgınlığımın", "içinde", ",", "eski", "İstanbul'un", "parke", "döşeli", "caddelerinde", "ydim", "."]
(1900, 

["Tabbule", ",", "maydanoz", "salatası", "dır", "."]
(1900, 36)
0.21865611
["Bunu", "yapmayanların", "yöneticileri", "şahsen", "sorumlu", "tutulacaklar", "."]
(1900, 49)
1.3971026
["Nasıl", "olur", "."]
(1900, 9)
0.3462084
["Teknik", "raporun", "deşifresi", "kısa", "süre", "içinde", "yaptırıldı", "."]
(1900, 64)
1.5843571
["Dün", "gece", ",", "insanlar", "hakkında", "bazen", "acele", "hüküm", "verdiğimi", "kavradım", "."]
(1900, 121)
2.0583327
["Babam", "kapıda", "hazır", "beni", "bekliyordu", "."]
(1900, 36)
0.872945
["on", "günde", "bir", "Diyarbakır'a", "giderdi", "."]
(1900, 36)
1.629927
["Vücudunda", "yanık", "ve", "kırıklar", "var", "."]
(1900, 36)
1.1113303
["Benim", "bilgime", "ve", "araştırmalarıma", "göre", "böyle", "bir", "şey", "yok", "."]
(1900, 100)
1.737596
["Kahramanlarınızın", ",", "kadın", "-", "erkek", "ilişkilerine", "dönük", "yönlerinde", "de", ",", "kulaklarınıza", "inanamıyor", "ve", "duyduklarınızla", "düş", "kırıklıklarına", "duçar", "oluyorsunuz", "."]
(1900, 

Excessive output truncated after 524288 bytes.

(1900, 841)
2.9186053
["Buz", "kesiyorum", "gitgide", ",", "aldırdığ

In [65]:
ygold = createygold(s)

1×49 LinearAlgebra.Transpose{Any,Array{Any,1}}:
 0  0  0  0  0  0  0  1  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  1  0

In [69]:
# ********* silmeeee
accuracy(reshape(model(createrelmatrix(s)), (length(s),length(s))), s.head)

0.5714285714285714